In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts-v2")

In [2]:
using LinearAlgebra, Combinatorics
using Printf
using Dates

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
#include("load_geometry_data.jl")        # low-level parsers FIRST
#include("GeometryDataLoader.jl")        # defines GeometryDataset
include("write_geometry_data.jl")
# --- include all modules ---
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("write_connectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")


using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .write_connectivity_data: save_connectivity_data
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all

# for mkpath
import Base.Filesystem: mkpath

In [3]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, -2.77453, -0.980944, -1.69904],
        [0.0, 0.0, 0.0, -3.39809],
        [-0.240281, -0.693632, -0.980944, -1.69904],
        [0.0, 0.0, -2.94283, -1.69904]
    ],
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, -2.77453, -0.980944, -1.69904],
        [0.0, 0.0, 0.0, -3.39809],
        [0.0, 0.0, -2.94283, -1.69904],
        [0.898137, 2.74372, -0.980944, -1.69904]
    ],
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, -3.39809],
        [-0.240281, -0.693632, -0.980944, -1.69904],
        [0.0, 0.0, -2.94283, -1.69904],
        [0.898137, 2.74372, -0.980944, -1.69904]
    ]
]

3-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, -2.77453, -0.980944, -1.69904], [0.0, 0.0, 0.0, -3.39809], [-0.240281, -0.693632, -0.980944, -1.69904], [0.0, 0.0, -2.94283, -1.69904]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, -2.77453, -0.980944, -1.69904], [0.0, 0.0, 0.0, -3.39809], [0.0, 0.0, -2.94283, -1.69904], [0.898137, 2.74372, -0.980944, -1.69904]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -3.39809], [-0.240281, -0.693632, -0.980944, -1.69904], [0.0, 0.0, -2.94283, -1.69904], [0.898137, 2.74372, -0.980944, -1.69904]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    subfolder = base_folder * "simplex$(s)/"
    mkpath(subfolder)

    bdypoints = bdypoints_all[s]

    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints, subfolder)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:
All geometry data saved to 'geom_output/new-data/simplex1/'
Geometry pipeline complete. Data saved to 'geom_output/new-data/simplex1/'.
Boundary points for simplex 2:
All geometry data saved to 'geom_output/new-data/simplex2/'
Geometry pipeline complete. Data saved to 'geom_output/new-data/simplex2/'.
Boundary points for simplex 3:
All geometry data saved to 'geom_output/new-data/simplex3/'
Geometry pipeline complete. Data saved to 'geom_output/new-data/simplex3/'.


In [5]:
geom = GeometryCollection(datasets);

four_simplices = [[1,2,3,4,5], [1,2,3,5,6], [1,3,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

[1, 2, 3]


In [6]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [ ]:
run_face_xi_matching(geom);

In [8]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 3.9604086946210244e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 1.0648998194100275e-15).
✓ Closure condition satisfied (max residual = 1.9984014443252818e-15).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------

